In [3]:
import numpy as np
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

!git clone https://github.com/rodrigorivera/mds20_deepfolio
from mds20_deepfolio.models.NeuralHawkesProcess.DataWrapper import LOBDataset, collate_fn
from mds20_deepfolio.models.NeuralHawkesProcess.model import NHPModel

fatal: destination path 'mds20_deepfolio' already exists and is not an empty directory.


In [4]:
XLM_dataset = LOBDataset(np.load('/content/mds20_deepfolio/datasets/data/transfer/XLM.npy'))
XLM_loader = DataLoader(XLM_dataset, batch_size=12, collate_fn=collate_fn)

len(XLM_dataset)

65

In [6]:
from sklearn.metrics import accuracy_score
model = NHPModel(256, device=device).to(device)
model.load_state_dict(torch.load('/content/mds20_deepfolio/models/NeuralHawkesProcess/weights/NHP,all,256.pth'))

loss_time, type_acc,loss_llh,loss_event = 0, 0, 0, 0
for event_seq, time_seq in XLM_loader:

        event_seq = event_seq.to(device)
        time_seq = time_seq.to(device)
        
        intens, time, event = model.forward(event_seq, time_seq)

        loss_llh += (model.LogLikelihoodLoss(intens, time_seq) / (time_seq.shape[0] * time_seq.shape[1])).detach().cpu().numpy()
        loss_time += model.time_loss(time, time_seq).detach().cpu().numpy()
        loss_event += model.event_loss(event, event_seq).detach().cpu().numpy()
        type_acc += accuracy_score(event[:,:-1].argmax(dim=2).cpu().reshape(-1), 
                                                      event_seq[:, 1:].cpu().reshape(-1))
        


In [8]:
print('LogLike score:', -loss_llh/len(XLM_loader))
print('Time RMSE score:', loss_time/len(XLM_loader)**0.5)
print('Type prediction accuracy:', type_acc/len(XLM_loader))

LogLike score: -15.231625398000082
Time RMSE score: 53781.01114502642
Type prediction accuracy: 0.7039083333333332
